In [53]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [54]:
print(tf.__version__)

2.15.0


In [55]:
from tqdm.notebook import tqdm, trange
import io
from PIL import Image

In [56]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [57]:
import glob
img_folder = "./val2017/"
img_paths = sorted(glob.glob(img_folder+'/*'))
# print(img_paths)
print("Number of imgs in the folder:", len(img_paths))

Number of imgs in the folder: 5


In [72]:
# label
label_path = './data/ImageNetLabels.txt'
with open(label_path, "r", encoding="UTF8") as lbfile:
    labels = lbfile.read().splitlines()

# ground truths
# gt_path = './data/caffe_clsloc_validation_ground_truth.txt'
gt_path = './data/ILSVRC2012_validation_ground_truth.txt'
with open(gt_path,"r") as lbfile:
    lines = lbfile.readlines()
    gts = []
    for x in lines:
        print(x.split(' ')[1])
        gts.append(int(x.split(' ')[1].splitlines()[0]))
# gts = np.array(gts) + 1
gts = np.array(gts)

IndexError: list index out of range

In [40]:
model_folder = "./image_classifiers"
model_name = "efficientnet_b0_classification_1" # https://tfhub.dev/tensorflow/efficientnet/b0/classification/1
img_height,img_width = 224, 224
model_path = os.path.join(model_folder, model_name)
classifier = tf.keras.models.load_model(model_path)
# classifier = keras.layers.TFSMLayer(model_path, call_endpoint="serving_default")
classifier._name = model_name

In [41]:
from tensorflow.python.keras import backend
classifier.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')])

In [42]:
classifier.trainable = False

# Joint AE PNC

In [43]:
model_save_folder = "./joint_ae"

In [44]:
encoder_pnc = keras.models.load_model(model_save_folder + "/best_model_save_encoder")
decoder_pnc = keras.models.load_model(model_save_folder + "/best_model_save_decoder")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer Orthogonal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [52]:
# step 1
filenames = tf.constant(img_paths)
tf_labels = tf.constant(gts)

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, tf_labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
#     print(filename)
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    image /= 255.0
    ##################Start AE##################
#     image = encoderDecoder(image)
    ##################End of AE##################
    image = tf.image.resize(image, (img_height, img_width))
#     image=tf.expand_dims(image,0)
    return image, label

dataset = dataset.map(_parse_function)
# dataset = dataset.batch(32)


tf.Tensor([ 65 970 230 ... 232 982 355], shape=(50000,), dtype=int64)


ValueError: Dimensions 5 and 50000 are not compatible

In [14]:
# representative_dataset
def representative_dataset():
  for data in dataset.batch(1).take(5000):
#     print(data)
    yield [tf.dtypes.cast(data[0], tf.float32)]

# quantization settings
converter_int8 = tf.lite.TFLiteConverter.from_keras_model(encoder_pnc)
converter_int8.optimizations = [tf.lite.Optimize.DEFAULT]
converter_int8.representative_dataset = representative_dataset
converter_int8.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_int8.inference_input_type = tf.uint8  # or tf.uint8
converter_int8.inference_output_type = tf.uint8  # or tf.uint8
tflite_quant_model = converter_int8.convert()

INFO:tensorflow:Assets written to: /var/folders/th/8yt90_9d1375qgz708zr3hd80000gn/T/tmpu8gzb0qr/assets


INFO:tensorflow:Assets written to: /var/folders/th/8yt90_9d1375qgz708zr3hd80000gn/T/tmpu8gzb0qr/assets
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-02-27 13:09:26.645186: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-27 13:09:26.645199: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-27 13:09:26.645713: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/th/8yt90_9d1375qgz708zr3hd80000gn/T/tmpu8gzb0qr
2024-02-27 13:09:26.646122: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-27 13:09:26.646127: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/th/8yt9

NameError: name 'dataset' is not defined

In [15]:
# Save the model.
directory = './saved_tflite_models_demo/'

In [48]:
import os
if not os.path.exists(directory):
    os.makedirs(directory)
with open(directory+'best_encoder_tuned_model_uint8.tflite', 'wb') as f:
  f.write(tflite_quant_model)

NameError: name 'tflite_quant_model' is not defined

In [23]:
import sys
import argparse
import time
import numpy as np
import PIL.Image as Image
import tensorflow as tf

In [24]:
from pprint import pprint
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=directory+'best_encoder_tuned_model_uint8.tflite', num_threads=12)
# Get input and output tensors.
input_details = interpreter.get_input_details()
minp = interpreter.get_input_details()[0]['index']
output_details = interpreter.get_output_details()
mout=interpreter.get_output_details()[0]['index']
# input details
pprint(input_details)
# output details
pprint(output_details)

[{'dtype': <class 'numpy.uint8'>,
  'index': 0,
  'name': 'serving_default_input_3:0',
  'quantization': (0.003921568859368563, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.00392157], dtype=float32),
                              'zero_points': array([0], dtype=int32)},
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
  'sparsity_parameters': {}}]
[{'dtype': <class 'numpy.uint8'>,
  'index': 8,
  'name': 'StatefulPartitionedCall:0',
  'quantization': (0.0212536808103323, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.02125368], dtype=float32),
                              'zero_points': array([0], dtype=int32)},
  'shape': array([ 1, 32, 32, 10], dtype=int32),
  'shape_signature': array([-1, 32, 32, 10], dtype=int32),
  'sparsity_parameters': {}}]


In [25]:
interpreter.allocate_tensors()

In [26]:
def qprint(msg):
    """Print right away"""
    sys.stdout.write(msg + '\n')
    sys.stdout.flush()


def predict(img):
    imsz = img.size
    minsz = np.minimum(imsz[0], imsz[1])
    imsz = (np.int16(list(imsz))-minsz)//2
    crop = [imsz[0], imsz[1], imsz[0]+minsz, imsz[1]+minsz]

    img = np.asarray(img.resize((img_height,img_width),
#                                 box=crop,
#                                 resample=Image.LANCZOS
                               ),
#                     dtype="float32",
                    )[np.newaxis, ...]
#     print(img)
#     img = (img).astype('uint8')
#     img = img/255.0
#     print(img)
    interpreter.set_tensor(minp, img)
    tict = time.time()
    interpreter.invoke()
    tict = time.time()-tict
#     print(tict)
    return interpreter.get_tensor(mout)


In [19]:
from tqdm.notebook import tqdm, trange

quant_num_imgs = 10000

In [20]:
_,de_h,de_w,de_c = output_details[0]['shape']
quant_img_paths, quant_img_gts = img_paths[-quant_num_imgs:], gts[-quant_num_imgs:]

In [20]:
# test_buffer = np.zeros((quant_num_imgs, de_h, de_w, de_c))
# print(test_buffer.shape)
# for idx, (cimg, cgt) in tqdm(enumerate(zip(quant_img_paths, quant_img_gts)), total=len(quant_img_paths)):
#     with Image.open(cimg).convert('RGB') as im:
#         im = im.resize((img_height,img_width))
#         pr = predict(im)
#         test_buffer[idx] = pr

In [21]:
# np.save(os.path.join(directory, "test_buffer"), test_buffer)

In [21]:
test_buffer = np.load(os.path.join(directory, "test_buffer.npy"))

## Further Quantize into Lower Bits

In [22]:
def quantize(d, Q_level):
    d_min, d_max = np.amin(d), np.amax(d)
    d_min, d_max = 0,255
    q = np.digitize(d, np.linspace(d_min, d_max, Q_level), right=True)
    s = d_max - d_min
    z = d_min
    return q, s, z, Q_level


def de_quantize(q, s, z, Q_level):
    d = [e/Q_level*s+z for e in q]
    return np.array(d)

## Huffman Emcoding Added

In [23]:
from dahuffman import HuffmanCodec

def quantize_embedding_with_customized_codec(embed, Q_level, codec):
    ## Data flow:   embed -> quantize -> huffman encode -> huffman decode -> de_quantize
    enc_q, s, z, _ = quantize(embed, Q_level)
    
    enc_huff = codec.encode([str(e) for e in enc_q.flatten()])
    enc_huff_dec = [ int(x) for x in codec.decode(enc_huff)]
    #print("Compression size before/after and error:", len(embed), len(enc_huff), np.sum(np.abs(enc_huff_dec - enc_q)))

    enc_q_dq = de_quantize(enc_huff_dec, s, z, Q_level).reshape(embed.shape)
    #print("Mean quantization error and mean embed value:", np.mean(np.abs(embed - enc_q_dq)), np.mean(np.abs(embed)))
    return enc_q_dq, len(enc_huff)



In [26]:
# num_huffman_train_imgs = 10000
# encoded_shape = output_details[0]['shape']
# huffman_train_input = np.zeros((num_huffman_train_imgs,encoded_shape[1],encoded_shape[2],encoded_shape[3]))
# for idx, huffman_train_img in tqdm(enumerate(img_paths[:num_huffman_train_imgs]), total=num_huffman_train_imgs):
#     with Image.open(huffman_train_img).convert('RGB') as im:
#         im = im.resize((img_height,img_width))
#         pr = predict(im)

#         huffman_train_input[idx] = pr
# huffman_train_input.shape

In [27]:
# np.save(os.path.join(directory, "huffman_train_input_10000"), huffman_train_input)

In [24]:
huffman_train_input = np.load(os.path.join(directory, "huffman_train_input_10000.npy"))

# Feat Huffman

In [25]:
n_bit = 6
pixel_range = 2 ** n_bit

In [46]:
huffman_codec_list = []
for i in range(10):
    print("Generate code table for {}".format(i))
    training_list = (huffman_train_input[...,i].copy()).flatten()
    training_list = np.hstack((training_list, np.arange(0,256)))
    print(training_list.shape)
    enc_q, s, z, _ = quantize(training_list, pixel_range)
    print(enc_q)
    codec = HuffmanCodec.from_data([str(e) for e in enc_q])
    huffman_codec_list.append(codec)

for i, codec in enumerate(huffman_codec_list):
    codec.save(os.path.join(directory, 'huffman_table_fine_stoch_{}_bit_{:01d}'.format(n_bit, i)))

Generate code table for 0
(10240256,)
[37 40 41 ... 63 63 63]
Generate code table for 1
(10240256,)
[21 29 30 ... 63 63 63]
Generate code table for 2
(10240256,)
[22 23 23 ... 63 63 63]
Generate code table for 3
(10240256,)
[17 17 17 ... 63 63 63]
Generate code table for 4
(10240256,)
[13 12 12 ... 63 63 63]
Generate code table for 5
(10240256,)
[ 5  8  9 ... 63 63 63]
Generate code table for 6
(10240256,)
[14  8  7 ... 63 63 63]
Generate code table for 7
(10240256,)
[ 1  5  6 ... 63 63 63]
Generate code table for 8
(10240256,)
[ 8  5  5 ... 63 63 63]
Generate code table for 9
(10240256,)
[ 0  2  2 ... 63 63 63]


In [26]:

huffman_codec_list = []
for i in range(10):
    codec = HuffmanCodec.load(os.path.join(directory, 'huffman_table_fine_stoch_{}_bit_{:01d}'.format(n_bit, i)))
    huffman_codec_list.append(codec)

In [2]:
### Calculate accuracy at given size limits
num_test_imgs = 2000
k_list = [1,2,3,4,5,6,7,8,9,10]
Ks = len(k_list)
proposed_size_list = np.array(np.arange(0.5, 6.25, 0.25)) * 1024


feature_size_table= np.zeros((num_test_imgs, Ks+1))
feature_accuracy_table = np.zeros((num_test_imgs, Ks+1))

cumulative_true = np.zeros(len(proposed_size_list))

huffman_num_imgs = 10000
huffman_img_paths, huffman_img_gts = img_paths[-huffman_num_imgs:], gts[-huffman_num_imgs:]

for img_idx in tqdm(range(num_test_imgs)):
    pr = np.copy(test_buffer[-huffman_num_imgs+img_idx])
    gt_img = huffman_img_gts[-huffman_num_imgs+img_idx]
    pr = pr.astype('uint8')
    pr = pr[np.newaxis,...]
    
    pr_huffman = np.zeros_like(pr)
    huffman_size = 0
    pr_enc_q, s, z, _ = quantize(pr, pixel_range)
    pr_enc_q_dq = de_quantize(pr_enc_q, s, z, pixel_range)
    all_feature_stack = np.repeat(pr_enc_q_dq, Ks, axis=0)
#     print(all_feature_stack.shape)
    for j in range(Ks):
        pr_flat = pr[:,:,:,j]
        pr_huffman_feature, huffman_size_feature = quantize_embedding_with_customized_codec(pr_flat, pixel_range, huffman_codec_list[j])
        pr[:,:,:,j] = pr_huffman_feature
        huffman_size += huffman_size_feature
        feature_size_table[img_idx, j+1] = huffman_size # size table update
        all_feature_stack[j, :, :, j+1:] = 0
    all_feature_stack = all_feature_stack * output_details[0]['quantization'][0]
    decoded_result = decoder_pnc.predict(all_feature_stack)

    predictions = classifier.predict(decoded_result)
#     pprint(predictions.shape)
    
    depth = predictions.shape[1]
    gt_one_hot = tf.one_hot(gt_img, depth)[np.newaxis, ...]
    gt_one_hot = np.repeat(gt_one_hot, Ks, axis=0)
#     print(gt_img)
    accuracy_top_five = tf.keras.metrics.top_k_categorical_accuracy(
        gt_one_hot, predictions, k=5
    )
    feature_accuracy_table[img_idx, 1:] = accuracy_top_five
    
    sz_idx = np.searchsorted(feature_size_table[img_idx], proposed_size_list, side='left', sorter=None).astype(np.int32)

    cumulative_true = cumulative_true + np.take(feature_accuracy_table[img_idx], sz_idx-1)
#     print(cumulative_true)
print(cumulative_true / num_test_imgs)

np.save(os.path.join(directory, "feature_accuracy_table_{}_bit".format(n_bit)), feature_accuracy_table)
np.save(os.path.join(directory, "feature_size_table_{}_bit".format(n_bit)), feature_size_table)

NameError: name 'np' is not defined

In [57]:
proposed_size_list = np.array(np.arange(0.75, 6, 0.25)) * 1024

In [28]:
cumulative_true/num_test_imgs

array([0.    , 0.3815, 0.4505, 0.4515, 0.6835, 0.69  , 0.7465, 0.767 ,
       0.79  , 0.8065, 0.82  , 0.829 , 0.8335, 0.8405, 0.8495, 0.855 ,
       0.8595, 0.864 , 0.863 , 0.863 , 0.8625, 0.8625, 0.8625])